<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import shutil
from google.colab import files
from zipfile import ZipFile

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%%time
%cd ..
directory = '/content/cloned-repo'
if os.path.exists(directory):
  shutil.rmtree(directory)

!git clone -l -s git://github.com/kavyajeetbora/EAST.git /content/cloned-repo
%cd /content/cloned-repo
!ls

/
Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 234 (delta 18), reused 0 (delta 0), pack-reused 199
Receiving objects: 100% (234/234), 17.99 MiB | 23.86 MiB/s, done.
Resolving deltas: 100% (133/133), done.
/content/cloned-repo
dataset.py		     EAST_resnet.py		 model.py
detect.py		     EAST_resnet_training.ipynb  model_resnet.py
dsiplay_ground_truth.ipynb   EAST_training.ipynb	 train.py
EAST_detection.ipynb	     EAST_trial_run.ipynb
EAST_resnet_detection.ipynb  loss.py
CPU times: user 62.9 ms, sys: 19.8 ms, total: 82.8 ms
Wall time: 6.94 s


In [0]:
from dataset import *

In [5]:
def unzip_files(file,output_dir):
  with ZipFile(file, 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall(output_dir)
  print('Extracted',len(output_dir),'to output dir')

image_files = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Images/Images.Zip'
annotation_files = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotations/Annotations.Zip'

directory = 'Train Set'
if os.path.exists(directory):
  shutil.rmtree(directory)
  
os.mkdir('Train Set')
print('Created Directory')

Created Directory


In [0]:
image_folder = 'Train Set/Images'
label_folder = 'Train Set/Annotations'

In [6]:
%%time
unzip_files(image_files, output_dir='Train Set/Images')

Extracted 16 to output dir
CPU times: user 1.26 s, sys: 482 ms, total: 1.74 s
Wall time: 3.49 s


In [7]:
%%time
unzip_files(annotation_files, output_dir='Train Set/Annotations')

Extracted 21 to output dir
CPU times: user 521 ms, sys: 265 ms, total: 786 ms
Wall time: 1.27 s


In [8]:
print(len(os.listdir('Train Set/Images')))

5000


In [9]:
print(len(os.listdir('Train Set/Annotations')))

5000


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [11]:
%%time
from model import EAST
from loss import Loss

model = EAST()
pretrained_path = '/content/drive/My Drive/model_epoch_10.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
criterion = Loss()

Downloading: "http://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:25<00:00, 21.5MB/s]


CPU times: user 5.43 s, sys: 2.33 s, total: 7.76 s
Wall time: 34 s


## Training Full Batch 

In [0]:
torch.cuda.empty_cache()

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-4)
    #scheduler = lr_scheduler.StepLR(optimizer, step_size=epoch_iter//25, gamma=0.9)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      #scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        #break
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        # replace the weight file
        filename = '/content/drive/My Drive/model_epoch_{}.pth'.format(epoch)
        if os.path.exists(filename):
          os.unlink(filename)
        torch.save(state_dict, '/content/drive/My Drive/model_epoch_{}.pth'.format(epoch+1))
       
      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))

    return loss_curve

In [0]:
%%time
loss_curve = train(model, image_folder, label_folder, batch_size=4, lr=1e-4, epoch_iter=5, interval = 1)

5000
classify loss is 0.25605679, angle loss is 0.00219637, iou loss is 0.30294695
Epoch is [1/5], mini-batch is [1/1250], time consumption is 0.77799225, batch_loss is 0.58096743
classify loss is 0.23215955, angle loss is 0.02153082, iou loss is 0.33625993
Epoch is [1/5], mini-batch is [2/1250], time consumption is 0.67528749, batch_loss is 0.78372771
classify loss is 0.19979036, angle loss is 0.00275240, iou loss is 0.31656945
Epoch is [1/5], mini-batch is [3/1250], time consumption is 0.66452312, batch_loss is 0.54388386
classify loss is 0.37798780, angle loss is 0.03431792, iou loss is 0.46465322
Epoch is [1/5], mini-batch is [4/1250], time consumption is 0.66679907, batch_loss is 1.18582010
classify loss is 0.14713579, angle loss is 0.00140661, iou loss is 0.21290292
Epoch is [1/5], mini-batch is [5/1250], time consumption is 0.60629582, batch_loss is 0.37410480
classify loss is 0.19009703, angle loss is 0.00439486, iou loss is 0.25763485
Epoch is [1/5], mini-batch is [6/1250], ti

In [0]:
plt.plot(loss_curve)

1. [**Adam and AdamW algorithms**](https://www.fast.ai/2018/07/02/adam-weight-decay/)

2. [**The optimal learning rate during fine-tuning of an artificial neural network**](https://www.mikulskibartosz.name/the-optimal-learning-rate-during-fine-tuning-of-an-artificial-neural-network/)